In [2]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import time
import json
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
train_data = pd.read_parquet('s3://think-tank-casestudy/raw_data/train_data.parquet')
test_data = pd.read_parquet('s3://think-tank-casestudy/raw_data/test_data.parquet')